# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [8]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

### Required service

In [9]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# Custom
# icon_service = IconService(HTTPProvider("http://54.186.60.134:9000/", 3))

### NIDs

In [10]:
# Mainnet
# NID = 1
# Euljiro Testnet
# NID = 2
# Yeouido Testnet // Tbears
NID = 3
# Pagoda Testnet
# NID = 80

### A list for SCORE addresses 

In [14]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
SCORE_ADDRESS = "cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e"


# Mainnet
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [5]:
deployer_wallet = KeyWallet.load(<path to keystore file>,<password>)
caller_wallet = KeyWallet.load(<path to keystore file>,<password>)

SyntaxError: invalid syntax (<ipython-input-5-138e1b62e38d>, line 1)

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

In [12]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

hxfd8546d965c94071601c4fcd80ec1a02ae34c64a
hx3029e0412e2feaf49fd0a1cc66f3041c440682b9


### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [13]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('dice.zip'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x6d1d21c53dec4462a91094ab5e70115d53fe18493b54a7edace9828ce7e75af2',
 'blockHeight': 2232472,
 'blockHash': '0x83d879e911af7869d939f0db43431ab6fefe93496486e4790b116102233b037e',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
 'stepUsed': 1083405920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1083405920,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Updating contract

In [ ]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('dice.zip'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

### Calling a readonly method

In [17]:
_call = CallBuilder()\
    .from_(caller_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .method("get_game_status")\
    .build()
response = icon_service.call(_call)
print(response)
   

0x1


In [22]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Calling an external method

In [16]:
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS) \
    .nid(NID) \
    .nonce(100) \
    .method("toggle_game_status")\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hxfd8546d965c94071601c4fcd80ec1a02ae34c64a', 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e', 'stepLimit': '0x38770', 'timestamp': '0x5a6b849630924', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'toggle_game_status'}, 'signature': 'Hbi+rXNTb+ViTyroG2zx2bj40SaZ2dyN1TkBnyv7ptJ7f+7T0nt5UHf+4QhXUvmaemArCegtfVwOJ/ItHSiGlQA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xcd6257a87eba5171493de8a1d207879dc643925394381d9602ef2df1cb6e438a',
 'blockHeight': 2232518,
 'blockHash': '0xcd54b0f39593732b64733a10a6e780a1759b14e67455bba49185d68abc164db8',
 'txIndex': 1,
 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
 'stepUsed': 131280,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 131280,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

### ICX Transfer

In [18]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID) \
    .nonce(100) \
    .value(50*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hxfd8546d965c94071601c4fcd80ec1a02ae34c64a', 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e', 'stepLimit': '0x36ee8', 'timestamp': '0x5a6b84d0a2b14', 'nid': '0x3', 'value': '0x2b5e3af16b1880000', 'nonce': '0x64', 'signature': 'XEhvtYkb1Zww1DOj/Mlm/gp0FKSqxJeu/ywV/88Anzs30tfjtFbVSyxhVPPUTRUMZL1yrKT7tHLkcm2M6r4bngA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xe6fe79d33a3dd4629c14e3261edfa2714464c84df97fe7cabd6c88c1a1fae86e',
 'blockHeight': 2232548,
 'blockHash': '0xb443a475a8604dc833bd54d4b467577cb4dc07d52d0db9e8fcaa73190e7942ca',
 'txIndex': 2,
 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
 'stepUsed': 125000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 533760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [19]:
balance = icon_service.get_balance(SCORE_ADDRESS)
print(balance)

50000000000000000000


### Calling a payable method



In [21]:

params={
    "upper": 90,
    "lower": 1,
    "user_seed": "Lucky",
    "side_bet_amount": 10**18,
    "side_bet_type":"icon_logo1"
}
# Skip side_bet_amount and side_bet_type if you dont want to play sidebet

call_transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .value(10*10**18)\
    .method("call_bet")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, caller_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx3029e0412e2feaf49fd0a1cc66f3041c440682b9', 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e', 'stepLimit': '0x47f90', 'timestamp': '0x5a6b851691984', 'nid': '0x3', 'value': '0x8ac7230489e80000', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'call_bet', 'params': {'upper': '0x5a', 'lower': '0x1', 'user_seed': 'Lucky', 'side_bet_amount': '0xde0b6b3a7640000', 'side_bet_type': 'icon_logo1'}}, 'signature': 'vTEhpURGVODh4ccg95vhMeMvwOqwQPJHWUn7IUVYnSo6X4WG1hwAi7PMnnzwi/G78EIkGHsLHtqI9Spszhr1SQA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb65ea423fec534ed62bb452537dd83ac21d07682fb8482b68fa9164224d4da21',
 'blockHeight': 2232585,
 'blockHash': '0x06f85244e9e2b69485ad79ab2294a17c8d0bc513ac7de787eb5ef40b6cefd9c8',
 'txIndex': 1,
 'to': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
 'stepUsed': 194800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 194800,
 'eventLogs': [{'scoreAddress': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
   'indexed': ['BetSource(Address,int)',
    'hx3029e0412e2feaf49fd0a1cc66f3041c440682b9',
    '0x5a6b851691984'],
   'data': []},
  {'scoreAddress': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
   'indexed': ['BetResult(int,str,int)',
    '0x41',
    'Main bet win Side bet lose',
    '0x88b23acffd990000'],
   'data': []},
  {'scoreAddress': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
   'indexed': ['PayoutAmount(int,int,int)',
    '0x88b23acffd990000',
    '0x88b23acffd990000',
    '0x0'],
   'data': []},
  {'scoreAddress': 'cxf3fab011fdc27211ad89a70bd2fc1ccf99917a2e',
   'i